# Midproject: Find the next Hype!?
Team: Sammy and Meron

### Goal

In this project we will ....

Importing all necessary libraries and reading the data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import datetime
pd.set_option('display.max_columns', None) # displays all columns in dfs

import warnings
warnings.filterwarnings('ignore')

In [2]:
stockx = pd.read_excel("midprojectfinal.xlsx")
stockx.head(2)

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California


In [3]:
stockx.rename(columns=lambda col: col.replace(' ', '_').lower(), inplace=True)

In [4]:
#We decided not to separate the color from the model as it is an indespensable part of the model

stockx["sneaker_name"].value_counts()

adidas-Yeezy-Boost-350-V2-Butter                     11423
Adidas-Yeezy-Boost-350-V2-Beluga-2pt0                10395
Adidas-Yeezy-Boost-350-V2-Zebra                      10110
Adidas-Yeezy-Boost-350-V2-Blue-Tint                   9297
Adidas-Yeezy-Boost-350-V2-Cream-White                 9097
Adidas-Yeezy-Boost-350-V2-Sesame                      5553
adidas-Yeezy-Boost-350-V2-Static                      4983
Adidas-Yeezy-Boost-350-V2-Semi-Frozen-Yellow          4854
Air-Jordan-1-Retro-High-Off-White-University-Blue     4635
adidas-Yeezy-Boost-350-V2-Static-Reflective           3184
Nike-Air-Presto-Off-White-Black-2018                  1884
Nike-Air-Presto-Off-White-White-2018                  1883
Nike-Air-VaporMax-Off-White-2018                      1591
Nike-Blazer-Mid-Off-White-All-Hallows-Eve             1435
Nike-Blazer-Mid-Off-White-Grim-Reaper                 1398
Nike-Zoom-Fly-Off-White-Pink                          1359
Nike-Air-VaporMax-Off-White-Black                     13

In [5]:
#Cleaning up dataset and removing all blanks from brand
stockx["brand"] = stockx["brand"].apply(lambda x: x.strip())
stockx.head(2)

,order_date,brand,sneaker_name,sale_price,retail_price,release_date,shoe_size,buyer_region
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California


In [6]:
#Cleaning up dataset and creating identical naming 
stockx["sneaker_name"] = stockx["sneaker_name"].str.replace("adidas","Adidas")

In [7]:
stockx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99956 entries, 0 to 99955
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_date    99956 non-null  datetime64[ns]
 1   brand         99956 non-null  object        
 2   sneaker_name  99956 non-null  object        
 3   sale_price    99956 non-null  float64       
 4   retail_price  99956 non-null  int64         
 5   release_date  99956 non-null  datetime64[ns]
 6   shoe_size     99956 non-null  float64       
 7   buyer_region  99956 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(3)
memory usage: 6.1+ MB


In [8]:
#Dimensions of the dataset
stockx.shape

(99956, 8)

In [9]:
stockx["brand"].value_counts()

Yeezy        72162
Off-White    27794
Name: brand, dtype: int64

##### Checking for Nans

In [10]:
stockx.isna().sum()

order_date      0
brand           0
sneaker_name    0
sale_price      0
retail_price    0
release_date    0
shoe_size       0
buyer_region    0
dtype: int64

Dataset has no nan values.

### Feature Engineering

#### Price Ratio

In [11]:
stockx["price_ratio"] = (stockx["sale_price"]/stockx["retail_price"]).round(2)
stockx.head(2)

,order_date,brand,sneaker_name,sale_price,retail_price,release_date,shoe_size,buyer_region,price_ratio
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California,4.99
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California,3.11


#### Time Since Release

In [12]:
stockx["time_since_release"] = (stockx["order_date"]-stockx["release_date"]).dt.days.astype("int64")
stockx.head(2)

,order_date,brand,sneaker_name,sale_price,retail_price,release_date,shoe_size,buyer_region,price_ratio,time_since_release
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California,4.99,342
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California,3.11,282


In [13]:
#Defining a function for extracting the features models, color and x from the column "sneaker_name"

def feature_addition(properties):
    for i in properties:
          stockx[i] = stockx["sneaker_name"].apply(lambda x : 1 if "i" in x.split("-") else 0) 

#### Sneaker 

In [ ]:
#how to account for v2 vs 350v2
model_names = ["Adidas","V2","airmax90","airmax97","zoom","presto","airforce","blazer","vapormax"]

stockx["model_names"] = stockx.apply(lambda x: feature_addition(model_names), axis=1) 

In [ ]:
stockx.head(2)

#### Colorway

In [ ]:
colorway = ["Black"] #need to white colorway

stockx["colorway"] = stockx.apply(lambda x: feature_addition(colorway), axis=1) 

In [ ]:
#colorway = ["Black"] #need to white colorway

#for i in colorway:
    #stockx[i] = stockx["sneaker_name"].apply(lambda x : 1 if "i" in x.split("-") else 0)

In [ ]:
stockx.head(2) #black is not show yet??!

#### State

In [ ]:
stockx["buyer_region"].sort_values()

In [ ]:
colorway = ["California","New York","Florida","Oregon"] #need to white colorway

stockx["colorway"] = stockx.apply(lambda x: feature_addition(colorway), axis=1) 

### EDA: Visualization and Descriptive Statistics 

In [ ]:
stockx_num = stockx.select_dtypes(include=np.number)

In [ ]:
stockx_cat = stockx.select_dtypes(include=np.object)
stockx_cat

In [ ]:
stockx.describe().round(1)

In [ ]:
#Correlations

#### Plotting the target variable

In [ ]:
px.histogram(stockx, x="sale_price", title="Distribution of sneaker prices")

#sneaker prices are positively skewed

In [ ]:
px.box(stockx, x="sale_price", title="Boxplot of sneaker prices")



In [ ]:
px.histogram(stockx,x="price_ratio", y="brand")
fig.show()

In [ ]:
#stockx.corr()

In [ ]:
plt.scatter(data['ic2'], data['ic3'])

In [ ]:
sns.boxplot(x="", data=data

In [ ]:
stockx.corr(method='pearson')

### Lasso Regression

We use lasso regerssion for identifiy the best possible features which we will later use in building our model

### Linear Models

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state=0)

In [ ]:
display(X_train.head(3)

#### Linear Regression

In [ ]:
y_pred_train = regressor.predict(X_train)
y_pred_train

In [ ]:
y_pred_test = regressor.predict(X_test)
y_pred_test

In [ ]:
Validation

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
print("--- Performances of the model ---")
print("the r2 score on train  :", regressor.score(X_train, y_train))
print("the r2 score on test  :", regressor.score(X_test, y_test))

In [ ]:
y_pred = regressor.predict(X)

In [ ]:
r2 = r2_score(y, y_pred)
mse = mean_squared_error(y, y_pred, squared=True)
rmse = mean_squared_error(y, y_pred, squared=False)
mae = mean_absolute_error(y, y_pred)

print("R2_score:", r2)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

In [ ]:
4.3 Function for Various Models

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV #in case we want to find Lasso's optimal alpha
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def model_application_and_evaluation(scaler, model, X_train, X_test, y_train, y_test):
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train,y_train)
    print("score on train set:", model.score(X_train, y_train))
    print("score on test set:", model.score(X_test, y_test))

In [ ]:
#defining different scalers
scaler1 = StandardScaler()
scaler2 = MinMaxScaler()
scaler3 = PolynomialFeatures(degree=2, interaction_only=True)

In [ ]:
#defining different models
baseline_regression = LinearRegression()
ridge = Ridge(alpha=0.3) #L2 regularizer #hyperparameters
lasso = Lasso (alpha=0.1) #L1 regularizer
knn = KNeighborsRegressor(n_neighbors=3)
mlp = MLPRegressor(random_state=1, max_iter=500)

In [ ]:
model_application_and_evaluation(scaler1, baseline_regression, X_train, X_test, y_train, y_test)


In [ ]:
model_application_and_evaluation(scaler2, baseline_regression, X_train, X_test, y_train, y_test)


In [ ]:
model_application_and_evaluation(scaler3, baseline_regression, X_train, X_test, y_train, y_test)


In [ ]:
df_cat =df.select_dtypes(include=np.object)
df_cat